In [38]:
import pickle
import string
import annoy

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import FastText
from tqdm import tqdm_notebook

import operator

In [2]:
checks = pd.read_csv("чековые данные.csv")
checks.drop(['name'], axis=1, inplace=True)

with open("Product_dict.pkl","rb") as f:
    product_dict = pickle.load(f)

checks.head()

C:\Progs\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,sale_date_date,contact_id,shop_id,product_id,product_sub_category_id,product_category_id,brand_id,quantity
0,2018-12-07,1260627,1455.0,168308.0,906.0,205.0,-1.0,"1,00"
1,2018-12-07,198287,279.0,134832.0,404.0,93.0,-1.0,"1,00"
2,2018-12-07,2418385,848.0,101384.0,404.0,93.0,-1.0,"1,00"
3,2018-12-07,1285774,1511.0,168570.0,906.0,205.0,-1.0,"1,00"
4,2018-12-07,1810323,1501.0,168319.0,906.0,205.0,-1.0,"1,00"


In [3]:
product_dict

{'168308': '(197312) Пакет-майка 25см х 45см 906',
 '134832': '(62448) Перекись водорода р-р наружн. 3% фл.полимерн. 100мл 404',
 '101384': '(72183) Салициловая кислота р-р спирт 2% фл 40мл N1 404',
 '168570': '(197309) Пакет 28см х 50см 906',
 '146960': '(111023) Пакет "Аптека Озерки" 28 х 35см 906',
 '128700': '(51806) Ранитидин тб 150мг уп N20 595',
 '52539': '(19416) Платки носовые "Зева Кидс" №10 721',
 '94270': '(66684) Пакет майка "Благодарим за покупку" (28+14) х 43см №1 -1',
 '77426': '(5605) Иглы Микро-Файн д/шприц-ручки 30G 0,3*8мм N100 759',
 '92475': '(114214) Дорзопт капли глазные 2% 5мл. фл.-кап. N3 575',
 '158643': '(181230) PL Шприц одноразовый 3-комп. 3мл №1 762',
 '158600': '(181232) PL Шприц одноразовый 3-комп. 5мл №1 762',
 '106230': '(43344) Аллапинин тб 25мг N30 738',
 '110629': '(57733) Корвалол капли д/приема внутрь 25мл 738',
 '47251': '(121286) PL Контейнер д/биопроб стер. 60мл с крыш и ложкой в инд.уп 711',
 '39811': '(116543) PL Салфетка антисептическая спи

In [4]:
# Создадим ключевую колонку: время + касса + магазин
checks['key_col'] = checks['sale_date_date'].apply(str) + "_" + checks['contact_id'].apply(str) + "_" + checks['shop_id'].apply(str)
checks.head()

,sale_date_date,contact_id,shop_id,product_id,product_sub_category_id,product_category_id,brand_id,quantity,key_col
0,2018-12-07,1260627,1455.0,168308.0,906.0,205.0,-1.0,"1,00",2018-12-07_1260627_1455.0
1,2018-12-07,198287,279.0,134832.0,404.0,93.0,-1.0,"1,00",2018-12-07_198287_279.0
2,2018-12-07,2418385,848.0,101384.0,404.0,93.0,-1.0,"1,00",2018-12-07_2418385_848.0
3,2018-12-07,1285774,1511.0,168570.0,906.0,205.0,-1.0,"1,00",2018-12-07_1285774_1511.0
4,2018-12-07,1810323,1501.0,168319.0,906.0,205.0,-1.0,"1,00",2018-12-07_1810323_1501.0


In [5]:
checks.dropna(inplace=True)
checks = checks[checks['sale_date_date'].str.startswith("20")]
checks.reset_index(inplace=True, drop=True)
checks['sale_date_date'] = pd.to_datetime(checks['sale_date_date'])
checks.sort_values('sale_date_date', inplace=True)

# Достаточно отсортировать и сделать трейн-тест, чтобы построить валидацию. Забьем на разорванные сессии
train, test = train_test_split(checks, test_size=0.3)

In [6]:
# Обучим простые контетные рекомендации

# Для фильтрации пунктуации
exclude = set(string.punctuation)
# Для приведения слов в начальной форме
morpher = MorphAnalyzer()

# Для фильтрации стоп-слов
sw = get_stop_words("ru")

def preprocess_txt(line):
    spls = "".join(i for i in str(line).strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

sentences = [preprocess_txt(k) for k in product_dict.keys()]

# Подготовим кандидатогенератор, который будет отдавать фильмы похожие по текстовому описанию на те, которые оенил пользователь
# Обучим Fasttext и заэмбедим фильмы
modelFT = FastText(sentences=sentences, vector_size=20, min_count=1, window=5)

# Для того, чтобы быстро находить айтемы положим эмбединги их тайтлов в ANN индекс
# Создадим объект индекса
ft_index_names = annoy.AnnoyIndex(20 ,'angular')

# Будем хранить соответствия не только id-> фильм, но и фильм-> id, чтобы быстрее находить эмбеддинги айтемов
reverse_index_map = {}
counter = 0

for name in tqdm_notebook(product_dict.keys()):
    n_ft = 0
    reverse_index_map[counter] = name
    vector_ft = np.zeros(20)
    # Каждое слово обернем в эмбеддинг
    for word in preprocess_txt(name):
        if word in modelFT.wv.key_to_index:
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    ft_index_names.add_item(counter, vector_ft)
    counter += 1

# 
ft_index_names.build(10)

C:\Users\olga_\AppData\Local\Temp/ipykernel_124/3277854769.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for name in tqdm_notebook(product_dict.keys()):


  0%|          | 0/30418 [00:00<?, ?it/s]

True

In [7]:
reverse_item_mapping = {v: k for k, v in product_dict.items()}

def recommend(items_list):
    current_vector = np.zeros(20)
    c = len(items_list)
    for iid in items_list:
        iname = product_dict[iid]
        if iname not in reverse_index_map:
            c -= 1
            continue
        current_vector += np.array(ft_index_names.get_item_vector[reverse_index_map[iname]])
    return [product_dict[reverse_index_map[i]] for i in ft_index_names.get_nns_by_vector(current_vector / c, 10)]

In [8]:
recommend(['52539', '110342'])

C:\Users\olga_\AppData\Local\Temp/ipykernel_124/3531675664.py:12: RuntimeWarning: invalid value encountered in true_divide
  return [product_dict[reverse_index_map[i]] for i in ft_index_names.get_nns_by_vector(current_vector / c, 10)]


['(106357) Салфетки EURON д/ухода за лежачими больными №64 755',
 '(62527) Плавикс табл. п.п.о. 75мг N28 738',
 '(42786) Арифон ретард таб. п/о 1,5мг №30 735',
 '(61845) Ноопепт таб.10мг №50 638',
 '(120617) Глюкофаж Лонг таб.пролонг.действ.1г №30 462',
 '(96641) Пустырника экстракт тб 14мг N50 613',
 '(68258) Проваг капс. №10 448',
 '(106123) Минеральная вода Пролом 1,5л ПЭТ 569',
 '(58685) Альфа Нормикс таб.200мг №12 596',
 '(27088) Бифидум БАГ 1000доз фл.12мл №1 600']

In [9]:
# Обучение эмбеддингов по последовательности
from gensim.models import Word2Vec

train['product_id'] = train['product_id'].apply(int).apply(str)

grouped = train.groupby('key_col')
sentences = []

C:\Users\olga_\AppData\Local\Temp/ipykernel_124/2304771833.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['product_id'] = train['product_id'].apply(int).apply(str)


In [10]:
# Сформируем последовательности

from tqdm import tqdm 
sentences = []
for group in tqdm_notebook(grouped.groups):
    products = grouped.get_group(group)['product_id'].values
    if len(products) < 3:
        continue
    sentences.append(list(products))

C:\Users\olga_\AppData\Local\Temp/ipykernel_124/2026112379.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for group in tqdm_notebook(grouped.groups):


  0%|          | 0/5458204 [00:00<?, ?it/s]

In [12]:
modelW2V = Word2Vec(sentences, vector_size=10)

In [13]:
sentences[0]

['99821', '52529', '82717', '134475']

In [17]:
modelW2V.wv.most_similar([1,2])

[('37651', 0.9376478791236877),
 ('54889', 0.9195696115493774),
 ('71821', 0.9067696332931519),
 ('140961', 0.900420606136322),
 ('60496', 0.897085428237915),
 ('63413', 0.8964385390281677),
 ('74585', 0.8845712542533875),
 ('60602', 0.8844577670097351),
 ('51925', 0.8841484785079956),
 ('115324', 0.8771871328353882)]

In [25]:
def recommend_w2v(items_list):
    current_vector = np.zeros(10)
    c = len(items_list)+1
    for iid in items_list:
        iname = product_dict[iid]
        if iname not in modelW2V.wv:
            c -= 1
            continue
        current_vector += np.array(modelW2V.wv[reverse_index_map[iname]])
    return [product_dict[i[0]] for i in modelW2V.wv.similar_by_vector(current_vector / c, 10)]

In [26]:
recommend_w2v(['99821', '138583', '45321', '134475'])

['(107901) Глюкоза р-р д/инфуз 5% бут. 200мл №28 434',
 '(96317) Ибупрофен сусп.д/приема внутрь 100мг/5мл фл.100мл д/детей 397',
 '(71579) Динамико таб.п.п.о.100мг №1 683',
 '(114492) Кординорм Кор таб.2,5мг №90 738',
 '(123319) Ван Тач Селект Тест-полоски №50х3 (промо) 452',
 '(71365) Крем "Женьшеневый" д/зрелой кожи 40мл -1',
 '(99872) Аскорбиновая кислота р-р д/и 5% амп 2мл N10 429',
 '(70318) Тобропт капли глазные 0,3% фл.-кап. 5мл 575',
 '(102173) Клеопатра масло антицеллюлитное массажное 150мл -1',
 '(186975) Адиарин пор.д/приг.сусп.д/приема внутрь 250мг пак.№8 597']

In [27]:
# Создадим метод, который по id продукта будет возвращать похожие на него
def recommend(product_id, num_sampled=10):
    n = 0
    for key, value in reverse_index_map.items():
         if product_id == value:
                n = key          
    if n not in reverse_index_map:
        return []
    return ft_index_names.get_nns_by_item(n, num_sampled)

In [28]:
recommend('214112')

[10117, 1834, 19808, 15573, 8360, 3270, 8192, 13297, 17349, 4933]

In [29]:
# Сделаем сразу рандомный кандидатогенератор (он нам пригодится для кандидатогенерации по субкатегории)
def sample_random(sample_set, num_sampled=10):
    return np.random.choice(sample_set, num_sampled, replace=True)

In [31]:
# Сделаем кандидатогенератор по последним купленным товарам из субкатегории

product2subcat = {}
subcat_dict = {}

for i in tqdm_notebook(train.index):
    subcats = train.loc[i, "product_sub_category_id"]
    product2subcat[train.loc[i, 'product_id']] = subcats
#     for subcat in subcats:
    if subcats not in subcat_dict:
        subcat_dict[subcats] = []
    subcat_dict[subcats].append(train.loc[i, 'product_id'])

C:\Users\olga_\AppData\Local\Temp/ipykernel_124/3963481202.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(train.index):


  0%|          | 0/14000000 [00:00<?, ?it/s]

In [32]:
# формируем перечень всех купленных товаров
product_unique = train['product_id'].unique()

In [33]:
# Создадим метод, который по последнему купленному товару будет возвращать набор кандидатов
# товаров из субкатегории товаров

def candidate_generator(product_id):

    candidates_ft = []
    if product_id is not None:
        candidates_ft = list(recommend(product_id))
    candidates_random = list(sample_random(product_unique))
    candidates_random = [int(i) for i in candidates_random]
    candidates_ft = [i for i in candidates_ft[1:]]
    candidates_subcat = []
    try:
        for subcat in product2subcat.get(str(product_id), []):
            candidates_subcat += list(sample_random(subcat_dict[subcat], 5))
    except:
        candidates_subcat = sample_random(subcat_dict[product2subcat.get(str(product_id), [])], 5)
    candidates_subcat = [int(i) for i in candidates_subcat]
    # Отфильтруем входной продукт (он мог попасть в кандидаты) и уберем дубликаты
    return set([i for i in candidates_ft + candidates_subcat + candidates_random if i!= product_id])

In [34]:
# Проверим как работает кандидатогенератор
candidate_generator(22453)

{4578,
 13107,
 13713,
 14380,
 17801,
 19602,
 19860,
 22703,
 23899,
 26925,
 28168,
 29073,
 54302,
 55276,
 75095,
 101264,
 125983,
 135298,
 136835,
 136850,
 141339,
 144398,
 154216,
 355759}

In [35]:
# частота покупки товара
product_freq = train['product_id'].value_counts().to_dict()

In [36]:
def recommend_multy_range(item_list):
    сandidates_list = [] # перечень сгенерированных товаров
    rec_items = [] # перечень рекомендованных товаров
    new_dict = {} # отранжированный словарь рекомендованных товаров 
    
    for iid in item_list:
#       получаем перечень кандидатов
        сandidates_list = list(candidate_generator(iid))
#         ранжируем кандидатов по частоте покупки товара
        for elem in сandidates_list:
            if str(elem) in product_freq.keys():
                new_dict[elem] = product_freq.get(str(elem))
        # сортируем товар по наболее часто покупаемому
    sorted_x = sorted(new_dict.items(), key=operator.itemgetter(1), reverse=True)
#   выдаем 10 наиболее часто покупаемых товаров
    for i in sorted_x[:10]:
        rec_items.append(product_dict[str(i[0])])
    return rec_items

In [39]:
recommend_multy_range(['120171', '69028', '62087'])

['(25205) Трентал таб. п.о. 400мг N20 738',
 '(49135) Камфорное масло 10% фл 30мл N1 648',
 '(89314) Кэа Вумен Прокладки гигиенические Найт Ультра Драй №7 555',
 '(16294) Прокладки "Molimed premium midi" женск. впитыв. 420 мл. №14 арт.168644\\1683990 555',
 '(15728) Дилтиазем Ланнахер тб п/о ретард 180мг N30 738',
 '(63495) Прокладки гигиенические Либресс Ультра Нормал с мягкой поверхностью №10 арт.8873 555',
 '(22095) Тампоны ватные "O.b." Pro Comfort супер № 16 арт.23432 558',
 '(45640) Тампоны Котекс Нормал уп №16 558',
 '(29826) Тампоны Котекс Нормал уп №8 558',
 '(15839) Тампоны ватные "O.b." супер коробка №16 арт.18458 558']

Для грамотной валидации нам не хватает дополнительных сведений, для какого набора болезней применяются эти препараты, остается ориентироваться только на категории товара. Также предсказания строятся только на предыдущих покупках одного препарата с другим. 
В целом, система крайне слаба для реального применения. 